### Build and push docker image from ECR 
* Get aws account related variables such as account id and region and compose the full docker image name 
* Dockerfile takes the fullname as a parameter to build docker image 
* Push to ecr

In [ ]:
%%bash
./vgg16-tl-tf22-build_and_push.sh

In [ ]:
%env fullname=230755935769.dkr.ecr.us-east-1.amazonaws.com/vgg16-tl-tf

In [ ]:
!echo $fullname 

In [ ]:
!pygmentize vgg16-tl-tf22-build_and_push.sh

In [ ]:
!pygmentize tf22-container/Dockerfile

In [ ]:
!pygmentize tf22-container/main.py

#### training locally 
* test container locally by dogs_and_cat.zip 

### Put your own training data set according to following structure - and you are good to go 
* `$ROOT_FOLDER/input/data/$label/*.jpg`

In [ ]:
!wget -O dogs_and_cat.zip "https://tinyurl.com/y3wmmygo"
!unzip dogs_and_cat.zip 

In [ ]:
!nvidia-docker run -it -v ${PWD}/dogs_and_cat:/opt/ml/ \
    $fullname  train 

In [ ]:
!aws s3 mb s3://ws-yolov4-yianc/ --region us-east-1

In [ ]:
!aws s3 cp --recursive dogs_and_cat s3://ws-yolov4-yianc/dogs_and_cat

In [ ]:
import boto3
region = boto3.session.Session().region_name
bucket = 'ws-yolov4-yianc'


In [ ]:
account=boto3.client('sts').get_caller_identity().get('Account')
repositoryUri="{}.dkr.ecr.{}.amazonaws.com/vgg16-tl-tf".format(account, region)
repositoryUri

In [ ]:
from datetime import datetime
now = datetime.now()
job_name = 'vgg16-' + now.strftime("%Y-%m-%d-%H-%M-%S")
job_name

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:

train='s3://{}/dogs_and_cat/input/data/train/'.format(bucket)
validation='s3://{}/dogs_and_cat/input/data/validation/'.format(bucket)
outpath='s3://{}/model/'.format(bucket)
(train, validation, outpath) 


In [ ]:
sm = boto3.client('sagemaker')

response = sm.create_training_job(
      TrainingJobName=job_name,
      AlgorithmSpecification={
          'TrainingImage': repositoryUri,
          'TrainingInputMode': 'File'
      },
      RoleArn=role,
      InputDataConfig=[
          {
              'ChannelName': 'train',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',
                      'S3Uri': train,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'validation',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': validation,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          
      ],
      OutputDataConfig={
          'S3OutputPath': outpath
      },
      ResourceConfig={
          'InstanceType': 'ml.p3.2xlarge',
          'InstanceCount': 1,
          'VolumeSizeInGB': 10,
      },
      StoppingCondition={
        'MaxRuntimeInSeconds': 60*60*5,
      }
  )
response